In [1]:
import pandas as pd
import numpy as np
import re
import string
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# 1. Load the dataset
file_path = "/mnt/data/Spam Email Detection - spam.csv"  # Uploaded file path
df = pd.read_csv("Spam Email Detection - spam.csv", encoding="latin-1")

# 2. Select relevant columns and rename them
df = df.iloc[:, :2]  # Keep only the first two columns
df.columns = ['label', 'message']  # Rename columns

# 3. Convert labels ('spam', 'ham') to binary (spam=1, ham=0)
df['label'] = df['label'].map({'spam': 1, 'ham': 0})

# 4. Function to clean text
def clean_text(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'\d+', '', text)  # Remove numbers
    text = text.translate(str.maketrans('', '', string.punctuation))  # Remove punctuation
    text = text.strip()  # Remove extra spaces
    return text

df['message'] = df['message'].apply(clean_text)

# 5. Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df['message'], df['label'], test_size=0.2, random_state=42)

# 6. Convert text into numerical features (TF-IDF Vectorization)
vectorizer = TfidfVectorizer(stop_words='english')
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# 7. Train the machine learning model (Naive Bayes)
model = MultinomialNB()
model.fit(X_train_tfidf, y_train)

# 8. Make predictions
y_pred = model.predict(X_test_tfidf)

# 9. Print results
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

# 10. Function to predict new emails
def predict_spam(email):
    email = clean_text(email)  # Clean the text
    email_tfidf = vectorizer.transform([email])  # Convert to features
    prediction = model.predict(email_tfidf)  # Predict
    return "Spam" if prediction[0] == 1 else "Not Spam"

# Test on a new email
print("\nTest Email Prediction:", predict_spam("Win a lottery now! Click here to claim your prize."))


Accuracy: 0.968609865470852

Confusion Matrix:
 [[965   0]
 [ 35 115]]

Classification Report:
               precision    recall  f1-score   support

           0       0.96      1.00      0.98       965
           1       1.00      0.77      0.87       150

    accuracy                           0.97      1115
   macro avg       0.98      0.88      0.93      1115
weighted avg       0.97      0.97      0.97      1115


Test Email Prediction: Spam
